In [1]:
# Ensure we are in the correct directory #
#----------------------------------------#

%cd ~/Documents/contabo/equity2/dashboard/playground/barycentres
%pwd

/Users/ludwigjonsson/Documents/contabo/equity2/dashboard/playground/barycentres


'/Users/ludwigjonsson/Documents/contabo/equity2/dashboard/playground/barycentres'

In [2]:
# Set up current notebook #
#-------------------------#

try: IS_MASTER
except: IS_MASTER = False

if not IS_MASTER:
    DATA_DIR = './data'
    RAW_FILE0 = '/path/to/raw/file.csv'
    PROCESSED_FILE = f'{DATA_DIR}/processed.pkl'
    OTHER_PREPROCESS_PARAM = ...


In [ ]:
import requests
import datetime
import urllib
import sqlite3
import yfinance as yf
import pandas as pd

In [ ]:
import requests
import datetime
import urllib
import sqlite3
import yfinance as yf
import pandas as pd

from bs4 import BeautifulSoup
from lxml import html

import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

from utils import dfsavecsv

@dfsavecsv(name='sp600', dir='ext_data')
def get_sp600_constit_wikipedia():
    '''
    Gets the S&P 600 (SmallCap) table from Wikipedia at
    'https://en.wikipedia.org/wiki/List_of_S%26P_600_companies'

    Returns it as a pandas DataFrame.
    '''
    sp600_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_600_companies'

    response = requests.get(sp600_url, headers={"User-Agent":"Mozilla/5.0"})
    logger.info(f'Wikipedia responded with status code: {response.status_code} for S&P600 table.')
    soup = BeautifulSoup(response.text, 'html.parser')
    

    table = soup.find('table',{'class': 'wikitable'})
    rows = table.find('tbody').find_all('tr')
    header = rows[0]
    body = rows[1:]

    col_names = [x.get_text().replace('\n', '') for x in header.find_all('th')]

    data = []
    for row in body:
        columns = [col.get_text().replace('\n', '') for col in row.find_all('td')]
        data.append(columns)

    df = pd.DataFrame(data, columns=col_names)
    logger.info(df)

    df = df[['Symbol', 'Company', 'GICS Sector', 'GICS Sub-Industry']]
    df.columns = ['symbol', 'company', 'industry', 'subindustry']

    return df